# Train Pytorch Model for Fashion MNIST using features and classifier methods

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils import data as D
import os
import numpy as np
from PIL import Image
from keras import datasets

In [2]:
from keras import datasets

Using TensorFlow backend.


In [7]:
dataset = 'mnist'

In [3]:
class Net(nn.Module):
    def __init__(self, features, num_classes, init_weights=True):
        super(Net, self).__init__()
        
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(4*4*50, 500),
            nn.ReLU(True),
            nn.Linear(500, num_classes)
        )
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        # x are the logits values
        return x 
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output = F.log_softmax(output, dim=1)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            output = F.log_softmax(output, dim=1)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
"""
torch.nn.Conv2d(in_channels, out_channels, kernel_size, 
stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
"""

def make_layers(cfg, in_channels, kernel_size, stride, padding, batch_norm=False):
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=kernel_size, padding=padding)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


"""
Refer VGG19_bn configurationh here: 
https://github.com/pytorch/vision/blob/76702a03d6cc2e4f431bfd1914d5e301c07bd489/torchvision/models/vgg.py#L63
"""
cfgs = {
    #'E': [64, 64, 'M',128, 128, 'M',256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',512, 512, 512, 512, 'M'],
    'E': [20, 'M', 50, 'M']
}

model_layers = make_layers(cfgs['E'],in_channels=1, kernel_size=5, stride=1, padding=0, batch_norm=False)

In [5]:
model_layers

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [6]:
class CustomDS(D.Dataset):
    """
    A customized data loader.
    """
    def __init__(self, path, train=True):
        """ Intialize the dataset
        """
        if train:
            data_path = os.path.join(path,'x_train.npy')
            targets_path = os.path.join(path,'y_train.npy')
        else:
            data_path = os.path.join(path,'x_test.npy')
            targets_path = os.path.join(path,'y_test.npy')

        self.path = data_path
        self.data = np.load(data_path)
        self.targets = np.load(targets_path)
        #self.transform = transforms.ToTensor()
        self.transform = transforms.Compose([
                       transforms.ToTensor()
                       #transforms.Normalize((0.1307,), (0.3081,))
                   ])
        self.len = np.shape(self.data)[0]
        
    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        data = self.data[index]
        image = Image.fromarray(data)
        
        target = int(self.targets[index])
        
        #data = (data * 255).astype(np.uint8)
        #data = data.reshape(28,28)
        #image = Image.fromarray((data * 255).astype(np.uint8))
        #image = Image.fromarray(data.astype(np.uint8))
        
        return self.transform(image), target

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [8]:
directory = './data/'+dataset
IS_DATA_READY = False

if not IS_DATA_READY:

    (x_train, y_train) ,(x_test, y_test) = datasets.mnist.load_data()

    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Normalize
    x_train = x_train / 255
    x_test = x_test / 255

    nb_classes = y_train.shape[0]

    #debug- Play around with MNIST here
    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))

    print('row, col, channels : {}'.format(x_train.shape[1:4]))
    print('num of classes: {}'.format(nb_classes))

    np.save(directory + '/x_train.npy', x_train)
    np.save(directory + '/y_train.npy', y_train)
    np.save(directory + '/x_test.npy', x_test)
    np.save(directory + '/y_test.npy', y_test)

else:
    x_train = np.load(directory + '/x_train.npy')
    y_train = np.load(directory + '/y_train.npy')
    x_test = np.load(directory + '/x_test.npy')
    y_test = np.load(directory + '/y_test.npy')
    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))


    

x_train shape : (60000, 28, 28)
y_train shape : (60000,)
x_test shape : (10000, 28, 28)
y_test shape : (10000,)
x_train shape : (60000, 28, 28)
y_train shape : (60000,)
x_test shape : (10000, 28, 28)
y_test shape : (10000,)
row, col, channels : (28, 28)
num of classes: 60000


In [9]:
type(x_train)

numpy.ndarray

In [10]:
x_train.shape

(60000, 28, 28)

In [11]:
np.max(x_train[0])

1.0

In [12]:
# Simple dataset. Only save path to image and load it and transform to tensor when call __getitem__.
filepath = './data/'+dataset
train_set = CustomDS(filepath, train=True)
test_set = CustomDS(filepath, train=False)

# total images in set
print(train_set.len)
print(test_set.len)

60000
10000


In [13]:
# main method
## Training settings
# input batch size for training (default: 64)
BATCH_SIZE = 64

# input batch size for testing (default: 1000)
TEST_BATCH_SIZE = 1000

# number of epochs to train
EPOCHS = 10

#learning rate (default: 0.01)
LR = 0.01

#SGD momentum (default: 0.5)
MOMENTUM = 0.5

# how many batches to wait before logging training status
LOG_INTERVAL = 10

SAVE_MODEL = True
SEED = 1
NO_CUDA = False
USE_CUDA = not NO_CUDA and torch.cuda.is_available()

NUM_CLASSES=10

torch.manual_seed(SEED)

device = torch.device("cuda" if USE_CUDA else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}


In [14]:
model = Net(model_layers, num_classes=NUM_CLASSES).to(device)

In [15]:
model

Net(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)

In [16]:
# Setup train and test loader

train_loader = D.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, **kwargs)
test_loader = D.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, **kwargs)


optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, optimizer, epoch, LOG_INTERVAL)
    test(model, device, test_loader)

model_dir = './model/'+dataset+'/v2'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if (SAVE_MODEL):
    print('saving model to ', model_dir+"/" + dataset+"_cnn.pt")
    torch.save(model.state_dict(),model_dir+"/"+dataset+"_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302175
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.301234
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.301275
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.299695
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.298930
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.297409
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.296603
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.295110
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.293866
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.292507
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.292564
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.292855
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.291507
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.286737
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.281012
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.282207
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.283581
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.281364
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.272802
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.269256
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.151394
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.246950
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.220777
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.175057
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.171170
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.057663
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.126021
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.184556
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.245359
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.106531
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.028386
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.234400
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.080027
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.106170
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.099878
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.193396
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.095779
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.142565
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.196927
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.051661


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.093544
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.086944
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.069968
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.093986
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.076378
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.101468
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.035767
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.049744
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.124116
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.150077
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.028734
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.105306
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.124380
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.094254
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.010100
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.074180
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.083422
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.160928
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.123916
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.011104


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.037619
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.113429
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.047734
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.088039
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.144166
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.016218
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.067804
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.071697
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.133504
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.076159
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.027372
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.062081
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.012271
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.057169
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.043336
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.106477
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.009857
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.016842
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.020880
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.024651


Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.002984

Test set: Average loss: 0.0572, Accuracy: 9808/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.018317
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.011761
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.030150
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.069906
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.043899
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.017671
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.008560
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.009380
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.046027
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.051603
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.068494
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.111661
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.050317
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.050120
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.035181
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.070237
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.069437
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.016439
Train Epoch: 8 [11

Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.064684
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.045914
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.059680
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.112216
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.060542
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.046049
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.019806
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.005609
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.033754
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.016883
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.140614
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.024729
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.003026
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.037776
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.105182
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.014207
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.031008
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.062319
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.031049
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.010544
